In [ ]:
##### imports

import requests
import numpy as np
import statsmodels.api as sm
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
scaler = MinMaxScaler()

In [ ]:
##### API CAll from KEY

def extract_api_call(sample_key):
    # Split key by "."
    key_list = sample_key.split(".")
    # Extract FlowRef
    flowRef = key_list[0]
    # Extract key 
    key = sample_key[len(key_list[0])+1:]
    # Build call string
    # API Part of URL
    api_part_generic = "https://api.statistiken.bundesbank.de/rest/data/"
    api_part_specific = flowRef+"/"+key
    api_call = api_part_generic + api_part_specific
    return api_call

In [ ]:
##### Function that takes in the api-call and performs a request and stores the data in a response_json

def auto_response_saver(api_call):
    #Define the header
    headers = {'accept': 'application/vnd.sdmx.data+json;version=1.0.0',}
    # Define parameters
    params = {'detail': 'full',}
    # Define full-sub
    full_response = '?detail=full'
    # Create full call
    full_call = api_call + full_response
    # Get the actual response
    response = requests.get(full_call, params=params, headers=headers,)
    # As json
    response_list = response.json()
    return(response_list)

In [ ]:
##### Take Response and return DF

def extract_response(Response, ColName):
    Response_Numbers = []
    VarKey = list(Response["data"]["dataSets"][0]["series"].keys())[0]

    for key in Response["data"]["dataSets"][0]["series"][VarKey]["observations"]:
        Response_Numbers.append(Response["data"]["dataSets"][0]["series"][VarKey]["observations"][key][0])

    Response_Numbers_PD = pd.DataFrame(Response_Numbers, columns = [ColName])    

    # General Timestamp ["data"]["structure"]["dimensions"]["observation"][0]["values"]
    Response_Timestamp = []

    for element in Response["data"]["structure"]["dimensions"]["observation"][0]["values"]:
        Response_Timestamp.append(element["id"])

    Response_Timestamp_PD = pd.DataFrame(Response_Timestamp, columns = ["time"])

    Response_DF = pd.concat([Response_Timestamp_PD,Response_Numbers_PD],axis=1).set_index("time")
    
    return Response_DF

In [ ]:
#####Variable Declaration

Keys_All = {
    
#"Gold" : "BBEX3.M.XAU.EUR.EA.AC.C06",
#"ALQ" : "BBDL1.M.DE.Y.UNE.UBA000.A0000.A01.D00.0.R00.A",
#"Zinsentwicklung" : "BBK01.SUD189",
#"Realzinssätze_Banken" : "BBK01.SUR106",
#"EZB_Einlagenfazilität" : "BBK01.SU0200",
#"HVPI" :"BBDP1.M.DE.N.HVPI.C.A00000.I15.A",
#"Effektivzinsen": "BBK01.SUD131Z",
#"Auftragseingang_Bau":"BBDY1.M.B30.Y.G800.R0400.A"    
"IR_eff_total" : "BBK01.SUD131Z",
"VOL_total" : "BBK01.SUD231",
"IR_eff_1" : "BBK01.SUD116",
"VOL_1" : "BBK01.SZD216",
"IR_eff_15" : "BBK01.SUD117",
"VOL_15" : "BBK01.SUD217",
"IR_eff_510" : "BBK01.SUD118",
"VOL_15" : "BBK01.SUD218",
"IR_eff_10" : "BBK01.SUD119",
"VOL_10" : "BBK01.SUD219"   
}

##### Create separate varables from dict

for key, value in Keys_All.items():
    exec(f"{key} = '{value}'")